<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#file-conversion" data-toc-modified-id="file-conversion-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>file conversion</a></span></li><li><span><a href="#convert-mp3-files-to-wav-format" data-toc-modified-id="convert-mp3-files-to-wav-format-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>convert mp3 files to wav format</a></span></li><li><span><a href="#remove-mp3-files-to-conserve-space" data-toc-modified-id="remove-mp3-files-to-conserve-space-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>remove mp3 files to conserve space</a></span></li><li><span><a href="#audio-split-on-silence" data-toc-modified-id="audio-split-on-silence-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>audio split on silence</a></span></li></ul></div>

In [2]:
# import packages for conversion
import os
from os import listdir
from os.path import isfile, join
os.environ['LIBROSA_CACHE_DIR'] = '/tmp/librosa_cache'
import librosa
import numpy as np
from tqdm import tqdm
from pathlib import Path
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import make_chunks
import ffmpeg
import pandas as pd
import re
from pydub.utils import which
import librosa 
import soundfile as sf
AudioSegment.converter = which("ffmpeg")

In [3]:
# define input(data) and output paths 
data = Path("\\train_audio")
output = (r"\\output")

In [6]:
# create target folders for output identical to input folder structure
for directory in Path(data).iterdir():
    ebird_code = directory.name
    os.makedirs(f'{output}/{ebird_code}')

# check out some files

In [18]:
# get info for file
myaudio = AudioSegment.from_file(filesArray[4])
channel_count = myaudio.channels    #Get channels
sample_width = myaudio.sample_width #Get sample width
duration_frames = len(myaudio)
duration_in_sec = len(myaudio) / 1000#Length of audio in sec
sample_rate = myaudio.frame_rate
vol = myaudio.dBFS

print( "sample_width=",sample_width )
print ("channel_count=", channel_count)
print ("duration_in_frames=", duration_frames )
print ("duration_in_sec=", duration_in_sec )
print ("frame_rate=", sample_rate)
print ("dbfs=", vol)


sample_width= 2
channel_count= 2
duration_in_frames= 36023
duration_in_sec= 36.023
frame_rate= 44100
dbfs= -33.74869741471482


# define conversion process

In [13]:
def audioConvert(path):
    # import file 
        sound = AudioSegment.from_file(path)
        # reassign target folder to name for save
        name = path.replace('train_audio', 'clipped_audio')
        #set all to mono for consistency
        sound = sound.set_channels(1)
        # set to predetermined framerate
        sound = sound.set_frame_rate(32000)
        #target
        tDb = -22.0
        #current
        fDb = sound.dBFS
        # process change in dBFS
        change_in_dBFS = tDb - fDb
        sound = sound.apply_gain(change_in_dBFS)
        # export to path 
        sound.export(name.replace(r'mp3', 'wav'), format="wav")

In [9]:
# gather all wav audio files
files = librosa.util.find_files(data, ext=['mp3']) 
filesArray = np.asarray(files)

In [14]:
# apply function to each filepath in array
for file in tqdm(files): 
    audioConvert(file)

100%|██████████████████████████████████████████████████████████████████████████| 21375/21375 [1:06:31<00:00,  5.36it/s]


# OPTIONAL remove mp3 files to conserve space

In [6]:
# gather all wav audio files
files = librosa.util.find_files(data, ext=['mp3']) 
filesArray = np.asarray(files)

In [8]:
for f in filesArray:
    os.remove(f)

In [39]:
# or 
for filename in os.listdir(data):
    if filename.endswith('.mp3'):
        os.remove(filename)